In [18]:
import numpy as np
import pymc as pm
import matplotlib.pyplot as plt
%matplotlib inline

In [75]:
#Reading in the data
data = np.load('MC_50_5_4.npz.npy')

In [76]:
data.shape

(50, 5, 4)

In [77]:
#Inferring the number of classes and data length
numData = data.shape[0]
numExperts = data.shape[1]
numClasses = data.shape[2]

In [78]:
#Since we know the class distribution is equal
alpha = [1, 1, 1, 1]
rho = pm.Dirichlet('rho',theta=alpha, plot=False)
rho_completedD = pm.CompletedDirichlet('rho_completed', rho, plot=False)
z = pm.Container([pm.Multinomial('z_%i' %i,n=1,p=rho, plot=False) for i in range(numData)])
lam = 10
lambda_mat = lam*np.eye( numClasses) + np.ones((numClasses, numClasses))
confMatList = []
confMatHatList = []
for idx in range(numExperts):
    confMat = pm.Container([pm.Dirichlet(name='theta_row_'+str(i)+str(idx), theta=lambda_mat[i,:], plot=False)\
                      for i in range(numClasses)])
    confMatHat = pm.Container([pm.CompletedDirichlet('theta_row_hat'+str(i)+str(idx),\
                                                     confMat[i], plot=False)\
                      for i in range(numClasses)])
    confMatList.append(confMat)
    confMatHatList.append(confMatHat)

In [79]:
confMatList = pm.Container(confMatList)
confMatHatList = pm.Container(confMatHatList)

In [80]:
@pm.observed
def mc_log_likelihood(value=data, z=z, confMatList=confMatHatList):
    """
    value - data
    z - observed classes
    confMat - confidence matrix
    """
    log_like = 0
    numData = data.shape[0]
    numExperts = data.shape[1]
    numClasses = data.shape[2]
    for data_idx in range(numData):
        dat = data[data_idx]
        observed_cls = z[data_idx]
        for expert_idx in range(numExperts):
            dat_exp = dat[expert_idx]
            confMat = confMatList[expert_idx]
            for class_idx_row in range(numClasses):
                for class_idx_column in range(numClasses):
                    log_like += observed_cls[class_idx_row]\
                    *np.sum(np.multiply(dat_exp ,np.log(confMat[class_idx_row])))
    return log_like

In [81]:
mod = pm.Model([rho, rho_completedD, z, confMatList, confMatHatList, mc_log_likelihood])

In [82]:
mcmc = pm.MCMC(mod)
mcmc.sample(iter=10000, burn=1000, thin=5)
pm.Matplot.plot(mcmc)

 [-----------------100%-----------------] 10001 of 10000 complete in 8038.3 sec

In [83]:
for e_idx in range(numExperts):
    print "Confusion Matrix for expert ",str(e_idx+1)
    confMatHat = confMatHatList[e_idx]
    for idx in range(numClasses):
        print np.mean(confMatHat[idx].trace()[:], axis=0)

Confusion Matrix for expert  1
[[ 0.85659685  0.02146113  0.11070047  0.01124155]]
[[ 0.02579119  0.82787302  0.0335672   0.11276859]]
[[ 0.02287433  0.01491114  0.94858034  0.0136342 ]]
[[ 0.02856769  0.01160149  0.01608913  0.94374169]]
Confusion Matrix for expert  2
[[ 0.59664259  0.18321612  0.0637488   0.15639249]]
[[ 0.129656    0.77531912  0.06876543  0.02625945]]
[[ 0.00354765  0.23035683  0.75196428  0.01413124]]
[[ 0.01779601  0.23104574  0.13848052  0.61267774]]
Confusion Matrix for expert  3
[[ 0.68115803  0.19699838  0.05717661  0.06466698]]
[[ 0.03211415  0.74050633  0.02187245  0.20550708]]
[[ 0.13424786  0.01367836  0.62712251  0.22495126]]
[[ 0.16582071  0.16117385  0.0180226   0.65498284]]
Confusion Matrix for expert  4
[[ 0.78708279  0.10548511  0.09308596  0.01434614]]
[[ 0.06807938  0.87772615  0.03303473  0.02115975]]
[[ 0.06599089  0.06733619  0.69012503  0.17654789]]
[[ 0.15971064  0.01844529  0.01866387  0.8031802 ]]
Confusion Matrix for expert  5
[[ 0.47786088

In [ ]:
Confusion Matrix for expert  1
[[ 0.97115385  0.00961538  0.00961538  0.00961538]
 [ 0.00961538  0.97115385  0.00961538  0.00961538]
 [ 0.00961538  0.00961538  0.97115385  0.00961538]
 [ 0.00961538  0.00961538  0.00961538  0.97115385]]
Confusion Matrix for expert  2
[[ 0.66666667  0.11111111  0.11111111  0.11111111]
 [ 0.11111111  0.66666667  0.11111111  0.11111111]
 [ 0.11111111  0.11111111  0.66666667  0.11111111]
 [ 0.11111111  0.11111111  0.11111111  0.66666667]]
Confusion Matrix for expert  3
[[ 0.53846154  0.15384615  0.15384615  0.15384615]
 [ 0.15384615  0.53846154  0.15384615  0.15384615]
 [ 0.15384615  0.15384615  0.53846154  0.15384615]
 [ 0.15384615  0.15384615  0.15384615  0.53846154]]
Confusion Matrix for expert  4
[[ 0.875       0.04166667  0.04166667  0.04166667]
 [ 0.04166667  0.875       0.04166667  0.04166667]
 [ 0.04166667  0.04166667  0.875       0.04166667]
 [ 0.04166667  0.04166667  0.04166667  0.875     ]]
Confusion Matrix for expert  5
[[ 0.625  0.125  0.125  0.125]
 [ 0.125  0.625  0.125  0.125]
 [ 0.125  0.125  0.625  0.125]
 [ 0.125  0.125  0.125  0.625]]